In [1]:
import config, labeling, model, predict, utils, dataset2

In [80]:
import importlib
importlib.reload(model)

<module 'Model' from '/home/farzanashfaque/Project/NER/src/Model.py'>

In [2]:
from labeling import Label
from model import EntityModel
import config
from dataset2 import preprocess_data
from utils import prepare_data, join_subtokens, expand_entity_dict, label_encoding
from predict import get_predictions
from time import time


vals_dict = preprocess_data()
text_train = vals_dict['text_train']
text_val = vals_dict['text_val']
text_test = vals_dict['text_test']
tags_train = vals_dict['tags_train']
tags_val = vals_dict['tags_val']
tags_test = vals_dict['tags_test']
entity_dict = vals_dict['entity_dict']

tag2idx, idx2tag = label_encoding(tags_train)
labeler = Label(text_train, entity_dict)
print('Starting dependency parsing...')
dep = labeler.dependency_parse()
a = time()
print('Starting labeling...')
labels = labeler.get_labels()
b = time()
print('Time taken for labeling: {}'.format(b-a))
print('Relabling...')
labels = labeler.relabel(labels, dep)
print('Preparing data...')
text_enc_train, mask_train, tags_enc_train = prepare_data(text_train, labels, tag2idx)
text_enc_val, mask_val, tags_enc_val = prepare_data(text_val, tags_val, tag2idx)
text_enc_test, mask_test, tags_enc_test = prepare_data(text_test, tags_test, tag2idx)

model = EntityModel().model()

  1%|          | 289/47959 [00:00<00:16, 2887.51it/s]

Parsing...


  0%|          | 6/38367 [00:00<10:46, 59.35it/s]

Starting dependency parsing...


  1%|          | 407/38367 [00:00<00:09, 4069.26it/s]

Starting labeling...


  0%|          | 0/38367 [00:00<?, ?it/s]

Time taken for labeling: 2.9276187419891357
Relabling...
Preparing data...


100%|██████████| 4796/4796 [00:08<00:00, 573.71it/s]
Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

In [ ]:
model2 = EntityModel().model()
history = model2.fit([text_enc_train, mask_train],
                     tags_enc_train,
                     batch_size=32,
                     epochs=config.EPOCHS,
                     validation_data=([text_enc_val, mask_val],tags_enc_val))

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


 534/1199 [============>.................] - ETA: 3:57 - loss: 0.1590 - accuracy: 0.9572 - precision_2: 0.9710 - recall_2: 0.9505

In [8]:
history = model.fit([text_enc_train, mask_train],
                     tags_enc_train,
                     batch_size=config.BATCH_SIZE,
                     epochs=config.EPOCHS,
                     validation_data=([text_enc_val, mask_val],tags_enc_val))

1199/1199 [==============================] - 476s 397ms/step - loss: 0.0245 - accuracy: 0.9931 - precision: 0.9935 - recall: 0.9927 - val_loss: 0.4979 - val_accuracy: 0.9273 - val_precision: 0.9277 - val_recall: 0.9270


In [15]:
text_pred, labels_pred = get_predictions(model, text_enc_train, mask_train, idx2tag)
entity_dict3 = utils.expand_entity_dict(entity_dict2.copy(), text_pred, labels_pred, threshold=5)

100%|██████████| 38367/38367 [00:00<00:00, 126474.40it/s]


In [7]:
labeler = Label(text_train, entity_dict2)
labels = labeler.get_labels()
labels = labeler.relabel(labels, dep)
text_enc_train, mask_train, tags_enc_train = prepare_data(text_train, labels, tag2idx)

100%|██████████| 38367/38367 [01:21<00:00, 471.75it/s]


In [13]:
import numpy as np
from sklearn.metrics import classification_report
from time import time


a = time()
preds = model.predict([text_enc_val, mask_val], batch_size=32)
b = time()
t = (b-a)/60
print('Time taken for prediction: {} mins'.format(t))
preds = np.argmax(preds, axis=2)
y = np.argmax(tags_enc_val, axis=2)

preds_ = []
y_ = []

for i in range(len(preds)):
    preds_.extend(preds[i])
    y_.extend(y[i])

Time taken for prediction: 0.7719708879788717 mins


In [5]:
print(classification_report(y_, preds_))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    186895
           1       0.89      0.43      0.58      6314
           2       0.48      0.10      0.16      5753
           3       0.00      0.00      0.00       129
           4       0.18      0.56      0.27        57
           5       0.47      0.19      0.27      6788
           6       0.86      0.98      0.91     96290
           7       0.17      0.16      0.17        62
           8       0.04      0.02      0.03      1797
           9       0.71      0.26      0.38      2859

    accuracy                           0.93    306944
   macro avg       0.48      0.37      0.38    306944
weighted avg       0.92      0.93      0.92    306944



In [14]:
print('iter 2, continuous')
print(classification_report(y_, preds_))

iter 2, continuous
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    186895
           1       0.53      0.43      0.47      6314
           2       0.48      0.11      0.18      5753
           3       0.00      0.01      0.00       129
           4       0.19      0.58      0.28        57
           5       0.47      0.20      0.28      6788
           6       0.86      0.96      0.90     96290
           7       0.17      0.18      0.17        62
           8       0.03      0.02      0.02      1797
           9       0.65      0.26      0.38      2859

    accuracy                           0.93    306944
   macro avg       0.44      0.37      0.37    306944
weighted avg       0.91      0.93      0.92    306944



In [12]:
print('iter 2, discrete')
print(classification_report(y_, preds_))

iter 2, discrete
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    186895
           1       0.54      0.42      0.47      6314
           2       0.48      0.10      0.17      5753
           3       0.00      0.00      0.00       129
           4       0.19      0.56      0.28        57
           5       0.48      0.19      0.27      6788
           6       0.86      0.96      0.91     96290
           7       0.15      0.16      0.15        62
           8       0.03      0.02      0.03      1797
           9       0.68      0.27      0.38      2859

    accuracy                           0.93    306944
   macro avg       0.44      0.37      0.37    306944
weighted avg       0.91      0.93      0.92    306944



In [10]:
idx2tag

{1: 'tim',
 2: 'O',
 3: 'eve',
 4: 'nat',
 5: 'gpe',
 6: 'per',
 7: 'org',
 8: 'art',
 9: 'geo',
 0: 'PAD'}

In [19]:
len(entity_dict3)

8696

In [12]:
c = 0
for i in range(len(labels)):
    for j in range(len(labels[i])):
        lab = labels[i][j]
        if lab!='O' and lab!='PAD':
            c+=1

In [13]:
c

54469

In [22]:
model2.save_weights('../model-weights/model2.h5')
model.save_weights('../model-weights/model.h5')

In [7]:
text = 'Hey everyone. My name is john doe. today we will learn about "something". Thats all.'

In [6]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['ner', 'tagger', 'lemmatizer', 'textcat'])

In [8]:
doc = nlp(text)
for sent in doc.sents:
    for token in sent:
        print(token.dep_)

intj
ROOT
punct
poss
nsubj
ROOT
compound
attr
punct
npadvmod
nsubj
aux
ROOT
prep
punct
pobj
punct
punct
nsubj
ROOT
attr
punct
